In [53]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import csv
import json

def readDict(filename):
    fileObject = open(filename, 'r', newline='', encoding='utf-8')
    dictObject = csv.DictReader(fileObject)
    array = []
    for row in dictObject:
        array.append(row)
    fileObject.close()
    return array

In [54]:
filename = 'bsci-employees.csv'
bsciEmployees = readDict(filename)

In [55]:
filename = 'orcid_data.csv'
orcidData = readDict(filename)

In [76]:
testRatio = 90
departmentTestRatio = 90
for employeeIndex in range(0, len(bsciEmployees)):
    matched = False
    for row in orcidData:
        name = row['givenNames'] + ' ' + row['familyName']
        #ratio = fuzz.ratio(name, bsciEmployees[employeeIndex][0])
        #partialRatio = fuzz.partial_ratio(name, bsciEmployees[employeeIndex][0])
        #sortRatio = fuzz.token_sort_ratio(name, bsciEmployees[employeeIndex][0])
        
        output = ''
        # the set ratio seems to do the best job of matching
        setRatio = fuzz.token_set_ratio(name, bsciEmployees[employeeIndex]['name'])
        if setRatio >= testRatio:
            output = str(setRatio) + ' ' + name + ' / ' + bsciEmployees[employeeIndex]['name']
            
            if row['department'] == '':
                output += " WARNING: no department given in ORCID"
            else:
                # carry out a secondary test to see if any of the departments listed in the BSCI page
                # are a good match to the department given in the ORCID record
                
                # expand the role JSON into a list of dictionaries
                roleDict = json.loads(bsciEmployees[employeeIndex]['role'])
                departmentMatch = False
                for department in roleDict:
                    setRatio = fuzz.token_set_ratio(department['department'], row['department'])
                    if setRatio > departmentTestRatio:
                        departmentMatch = True
                        output += ' ' + str(setRatio) + ' ' + row['department']
                if not departmentMatch:
                    output += ' WARNING: ' + row['department'] + ' less than ' + str(departmentTestRatio) + '% match to any dept.'
            print(output)
            matched = True
            foundOrcid = row['orcid']
            # We only care about the first good match to an ORCID record, kill the loop after that
            break
    if matched:
        bsciEmployees[employeeIndex]['orcid'] = foundOrcid
    else:
        bsciEmployees[employeeIndex]['orcid'] = ''      

100 Brandt Eichman / Brandt F. Eichman 100 Department of Biological Sciences
100 Julian Hillyer / Julian F. Hillyer 100 Department of Biological Sciences
100 Lauren Jackson / Lauren Parker Jackson 100 Biological Sciences
100 Carl Johnson / Carl H. Johnson 100 Biological Sciences 100 Biological Sciences
100 Jared Nordman / Jared Nordman 100 Biological Sciences
100 Lars Plate / Lars Plate 100 Chemistry and Biological Sciences
100 Antonis Rokas / Antonis Rokas 100 Department of Biological Sciences 100 Department of Biological Sciences
100 Ann Tate / Ann Tate 100 Biological Sciences
100 Laurence Zwiebel / Laurence J. Zwiebel 100 Biological Sciences 100 Biological Sciences
100 Allison Leich Hilbun / Allison Leich Hilbun 100 Biological Sciences
100 Tetsuya Mori / Tetsuya Mori 100 Biological Sciences
100 Elwood Mullins / Elwood Mullins 100 Biological Sciences
100 Peng Xu / Peng Xu 100 Department of Biological Sciences
100 Suzana Herculano-Houzel / Suzana Herculano-Houzel 100 Department of Psy

In [78]:
filename = 'bsci-employees-with-orcid.csv'
with open(filename, 'w', newline='') as csvFileObject:
    fieldnames = ['name', 'degree', 'category', 'orcid', 'role']
    writer = csv.DictWriter(csvFileObject, fieldnames=fieldnames)
    writer.writeheader()
    for row in bsciEmployees:
        writer.writerow(row)
